<a href="https://colab.research.google.com/github/anozele/Mask_RCNN/blob/master/webappforface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))
from flask import Flask
import threading
from flask import Flask, render_template, redirect, url_for

from flask_uploads import UploadSet, configure_uploads, IMAGES, patch_request_class

from flask import request
import cv2
import os
import sys
import matplotlib.pyplot as plt

import pickle
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode


from werkzeug import secure_filename
from flask_ngrok import run_with_ngrok

172.28.0.2


In [12]:
! pip install flask_ngrok

In [13]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 44.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.1
    Uninstalling scipy-1.3.1:
      Successfully uninstalled scipy-1.3.1


In [0]:
import numpy as np

In [0]:
from flask_socketio import SocketIO

In [0]:
folder="/content/test"
os.mkdir(folder)

In [10]:
from google.colab import files

uploaded = files.upload()



Saving face_database.pkl to face_database.pkl


In [0]:
import os,shutil

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
src ="/content/drive/My Drive/detection"
dst ="/content/"
shutil.move(src, dst) 
  

In [0]:
import FaceToolKit as ftk
import DetectionToolKit as dtk

In [6]:
app = Flask(__name__)
app.config['MAX_CONTENT_PATH']=10000000
app.config['UPLOAD_FOLDER']="dataset"
run_with_ngrok(app) 
verification_threshhold = 1.175
image_size = 160
v = ftk.Verification()
image_path="dataset/"
# Pre-load model for Verification
v.load_model("./models/20180204-160909/")
v.initial_input_output_tensors()
d = dtk.Detection()



W0905 13:13:59.143991 140563445639040 deprecation_wrapper.py:119] From /content/FaceToolKit.py:14: The name tf.GPUOptions is deprecated. Please use tf.compat.v1.GPUOptions instead.

W0905 13:13:59.153694 140563445639040 deprecation_wrapper.py:119] From /content/FaceToolKit.py:15: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0905 13:13:59.156836 140563445639040 deprecation_wrapper.py:119] From /content/FaceToolKit.py:15: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0905 13:13:59.185976 140563445639040 deprecation_wrapper.py:119] From /content/facenet/face.py:77: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.



Model directory: ./models/20180204-160909/
Metagraph file: model-20180204-160909.meta
Checkpoint file: model-20180204-160909.ckpt-266000


W0905 13:14:01.819772 140563445639040 meta_graph.py:935] The saved meta_graph is possibly from an older release:
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
W0905 13:14:01.821799 140563445639040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/queue_runner_impl.py:391: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0905 13:14:01.846059 140563445639040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0905 13:14:02.534771 140563445639040 deprecation_wrapper.py:119]

In [0]:
def img_to_encoding(img):
        image = plt.imread(img)
        aligned = d.align(image, False)[0]
        return v.img_to_encoding(aligned, image_size)
def distance(emb1, emb2):
        diff = np.subtract(emb1, emb2)
        return np.sum(np.square(diff))



app.config['MAX_CONTENT_PATH']=10000000
app.config['UPLOAD_FOLDER']="dataset"
app.config['UPLOAD_FOLDER1']="test"


file = open("face_database.pkl",'rb')
facedb = pickle.load(file)
def add_to_database(images,database):
    db={}
    identity = os.path.splitext(os.path.basename(images))[0]

    db[identity]=img_to_encoding(os.path.join('dataset',images))
    database.update(db)

    f=open('face_database.pkl', 'wb')
    pickle.dump(database, f)
    f.close()
    return database
    

def take_photo(filename, quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open("dataset/"+filename+".jpg", 'wb') as f:
    f.write(binary)
  return "Thankyou for registering"


def take_loginphoto(filename="2", quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open("test/"+filename+".jpg", 'wb') as f:
    f.write(binary)
  return ""


def upload_file(f):
        file=secure_filename(f.filename)
        f.save(os.path.join(app.config['UPLOAD_FOLDER'],file))
        add_to_database(file,facedb)
        
        return "Thankyou for registering"
      

def upload_testfile(f):
        file=secure_filename(f.filename)
        f.save(os.path.join(app.config['UPLOAD_FOLDER1'],"2.jpg"))
        return "Thankyou for registering"


@app.route("/")
def index():
    return render_template("index.html")


@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":
        cam=request.form.get("webcam")
        log=request.form.get("login")
        f = request.files['file']
        if cam:
            if os.path.exists("test/2.jpg"):
                os.remove("test/2.jpg")
                   
            file = open("face_database.pkl",'rb')
            facedb = pickle.load(file)
            take_loginphoto()
            encoding = img_to_encoding("test/2.jpg")
            min_dist = 1000
            for (name, db_enc) in facedb.items():
                dist = distance(encoding, db_enc)
                if min_dist > dist:
                    min_dist = dist
                    identity = name
                        
            if min_dist > verification_threshhold:
                webcam.release()
                return "<h1>you are not registered ,please register</h1>",render_template('signup.html',form=form)

            else:
        
                print ("it's " + str(identity) + " ")
                
                return "Go to room no 1/2"
        else:
            if os.path.exists("test/2.jpg"):
                os.remove("test/2.jpg")
                   
            file = open("face_database.pkl",'rb')
            facedb = pickle.load(file)
            upload_testfile(f)
            encoding = img_to_encoding("test/2.jpg")
            min_dist = 1000
            for (name, db_enc) in facedb.items():
                dist = distance(encoding, db_enc)
                if min_dist > dist:
                    min_dist = dist
                    identity = name
                        
            if min_dist > verification_threshhold:
                
                return "<h1>you are not registered ,please register</h1>",render_template('signup.html',form=form)

            else:
                accuracy=(verification_threshhold-min_dist)/verification_threshhold
                print ("it's " + str(identity) + " ")
            
                return "Go to room no 1/2"
        
    return     render_template("login.html")
    

@app.route("/register", methods=["GET", "POST"])
def register():
    if request.method == "POST":
        uname = request.form.get('uname')
        cam=request.form.get("Webcam")
        reg=request.form.get("Register")
        f = request.files['file']
        print(f)
        if cam:
               return( take_photo(uname))
                
        else:
                return  (upload_file(f))
                
        

        #return redirect(url_for("login"))
    return render_template("register.html")


@app.route('/upload', methods=["GET", "POST"])
def uploadfile():
   return render_template('upload.html')
	


In [0]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I0905 13:14:38.175159 140563445639040 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fe53007c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


I0905 13:14:42.805051 140562030114560 _internal.py:122] 127.0.0.1 - - [05/Sep/2019 13:14:42] "GET / HTTP/1.1" 200 -
I0905 13:14:43.449359 140562030114560 _internal.py:122] 127.0.0.1 - - [05/Sep/2019 13:14:43] "GET /favicon.ico HTTP/1.1" 404 -
I0905 13:14:46.197338 140562030114560 _internal.py:122] 127.0.0.1 - - [05/Sep/2019 13:14:46] "GET /register HTTP/1.1" 200 -


<FileStorage: 'oneshot.JPG' ('image/jpeg')>


...

I0905 13:23:00.666050 140562030114560 _internal.py:122] 127.0.0.1 - - [05/Sep/2019 13:23:00] "GET /login HTTP/1.1" 200 -


<IPython.core.display.Javascript object>

I0905 13:23:05.843773 140562030114560 _internal.py:122] 127.0.0.1 - - [05/Sep/2019 13:23:05] "POST /login HTTP/1.1" 200 -


it's manoj 


I0905 13:23:07.529179 140562030114560 _internal.py:122] 127.0.0.1 - - [05/Sep/2019 13:23:07] "GET /login HTTP/1.1" 200 -


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I0905 12:04:46.731946 140324111087360 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename="21", quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open("test/"+filename+".jpg", 'wb') as f:
    f.write(binary)
  return "uploadede"

In [11]:
take_photo()

<IPython.core.display.Javascript object>

'uploadede'